In [ ]:
import openmm
import openmmtools
import simtk.unit as unit
import pathlib
import endstate_correction
from endstate_correction.system import create_charmm_system
from openmm.app import CharmmParameterSet, CharmmPsfFile, CharmmCrdFile, PDBFile, Simulation
from rdkit import Chem
import numpy as np
from typing import List

system_name = 'ZINC00107550'              
path = pathlib.Path(endstate_correction.__file__).resolve().parent
hipen_testsystem = f"{path}/data/hipen_data"
path_to_psf = f"{hipen_testsystem}/{system_name}/{system_name}.psf"
path_to_crd = f"{hipen_testsystem}/{system_name}/{system_name}.crd"
psf = CharmmPsfFile(path_to_psf)
coord = CharmmCrdFile(path_to_crd)
params = CharmmParameterSet(
    f"{hipen_testsystem}/top_all36_cgenff.rtf",
    f"{hipen_testsystem}/par_all36_cgenff.prm",
    f"{hipen_testsystem}/{system_name}/{system_name}.str",
)
# create openmm simulation system
mm_system = psf.createSystem(params, nonbondedMethod=openmm.app.NoCutoff)


In [ ]:
def _set_torsion(simulation: Simulation, dihedral_indices:dict, torsion_idx:int, restraint, angle:float):
    print(restraint.getNumTorsions())
    print(restraint.getTorsionParameters(torsion_idx))
    dihedral_index_i, dihedral_index_j, dihedral_index_k, dihedral_index_l = dihedral_indices[torsion_idx]
    restraint.setTorsionParameters(torsion_idx, 
                                dihedral_index_i,
                                dihedral_index_j,
                                dihedral_index_k,
                                dihedral_index_l,
                                1, 
                                angle, 
                                k=20*unit.kilojoules_per_mole
                                )
    print(restraint.getTorsionParameters(torsion_idx))
    restraint.updateParametersInContext(simulation.context)

def _get_energy(simulation: Simulation):
    state = simulation.context.getState(getEnergy = True)
    return state.getPotentialEnergy() / unit.kilojoules_per_mole

def _get_position(simulation: Simulation):
    state = simulation.context.getState(getPositions=True)
    return state.getPositions()
    
def torsion_scan_1D(dihedral_indices:dict, ncycles:int, restraint, simulation:Simulation)-> List[float]:
    
    energies = []
    torsion_idx = 0
    dihedral_index_i, dihedral_index_j, dihedral_index_k, dihedral_index_l = dihedral_indices[torsion_idx]
    
    for i in range(ncycles):    
        angle_i = i*360/ncycles*unit.degree
        #print(f"{angle_i=}")
        _set_torsion(simulation, dihedral_indices, torsion_idx, restraint, angle_i)
        simulation.step(100)
        simulation.minimizeEnergy()
        energy = _get_energy(simulation)
        energies.append(energy)
    return energies

def torsion_scan_2D(dihedral_indices:dict, ncycles:int, restraint, simulation:Simulation)-> List[float]:
    
    torsion_idx_i = 0
    torsion_idx_j = 1
    energy_2d = []
    for cycle_i, angle_i in enumerate(np.linspace(-np.pi, np.pi, ncycles)):  
        simulation.context.setPositions(coord.positions)
        _set_torsion(simulation, dihedral_indices, torsion_idx_i, restraint, angle_i)
        energies = []
        for cycle_j, angle_j in enumerate(np.linspace(-np.pi, np.pi, ncycles)):    
            _set_torsion(simulation, dihedral_indices, torsion_idx_j, restraint, angle_j)
            simulation.minimizeEnergy(tolerance=1*unit.kilojoules_per_mole/unit.nanometer)
            energies.append(_get_energy(simulation))
            pos = _get_position(simulation)
            PDBFile.writeFile(simulation.topology, pos, file=open(f'mconfs/conf_{cycle_i}_{cycle_j}_{angle_i}_{angle_j}.pdb', 'w'))
        energy_2d.append(energies)
        
        
    return energy_2d


In [ ]:
with open("temp.pdb", "w") as outfile:
    PDBFile.writeFile(psf.topology, coord.positions, outfile)
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
# Create the RDKit molecule
rdkit_mol = Chem.MolFromPDBFile("temp.pdb", removeHs=False)
#rdkit_mol = Chem.AddHs(rdkit_mol)
AllChem.Compute2DCoords(rdkit_mol)
# Visualize the molecule
for i, atom in enumerate(rdkit_mol.GetAtoms()):
    # For each atom, set the property "molAtomMapNumber" to a custom number, let's say, the index of the atom in the molecule
    atom.SetProp("molAtomMapNumber", str(atom.GetIdx()))

img = Draw.MolToImage(rdkit_mol)
img


In [ ]:
# 2D torsion plot
import simtk.openmm as mm
import simtk.unit as unit
import openmmtools
from matplotlib.pylab import plt
import os

#os.removedirs('mconfs')
#os.mkdir('mconfs')
#dihedral_indices = [(11,10,7,9),(7,6,4,3)]
dihedral_indices = [(1,2,7,8), (7,8,9,17)]

# Create a TorsionForce to handle the torsion angle restraints
restraint = mm.PeriodicTorsionForce()
# Add the TorsionForce to the system
mm_system.addForce(restraint)

# Add the torsion angle restraints to the system
for dihedral_idx, (idx_i, idx_j, idx_k, idx_l) in enumerate(dihedral_indices):
    restraint.addTorsion(idx_i, idx_j, idx_k, idx_l, 1,0*unit.radians,1*unit.kilojoule_per_mole)

# Set up the integrator
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2*unit.femtoseconds)

# Set up the simulation object
simulation = mm.app.Simulation(psf.topology, mm_system, integrator)

# Set the positions and velocities of the particles
simulation.context.setPositions(coord.positions)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

# Perform a torsion angle scan
ncycles = 20
energies = torsion_scan_2D(dihedral_indices, ncycles, restraint, simulation)
import seaborn as sns
from matplotlib.pyplot import figure
figure(figsize=(15.0, 15.0), dpi=600)

axs = sns.heatmap(energies, annot=True, fmt=".1f",linewidth=.5)
axs.set_yticklabels([f"{v:.1f}" for v in np.linspace(-np.pi, np.pi, ncycles)])
axs.set_xticklabels([f"{v:.1f}" for v in np.linspace(-np.pi, np.pi, ncycles)])
axs.set_xlabel('')
plt.show()

In [ ]:
# 1D torsion plot
import simtk.openmm as mm
import simtk.unit as unit
import openmmtools
from matplotlib.pylab import plt

#dihedral_indices = [(11,10,7,9),(7,6,4,3)]
dihedral_indices = [(7,6,4,3)]
dihedral_indices = [(2,3,5,6)]


# Create a TorsionForce to handle the torsion angle restraints
restraint = mm.PeriodicTorsionForce()
# Add the TorsionForce to the system
mm_system.addForce(restraint)

# Add the torsion angle restraints to the system
for dihedral_idx, (idx_i, idx_j, idx_k, idx_l) in enumerate(dihedral_indices):
    restraint.addTorsion(idx_i, idx_j, idx_k, idx_l, 1,0*unit.radians,1*unit.kilojoule_per_mole)

# Set up the integrator
integrator = mm.LangevinIntegrator(300*unit.kelvin, 1/unit.picosecond, 2*unit.femtoseconds)

# Set up the simulation object
simulation = mm.app.Simulation(psf.topology, mm_system, integrator)

# Set the positions and velocities of the particles
simulation.context.setPositions(coord.positions)
simulation.context.setVelocitiesToTemperature(300*unit.kelvin)

# Perform a torsion angle scan
ncycles = 40
energies = torsion_scan_1D(dihedral_indices, 60, restraint, simulation)
plt.plot(energies)
plt.show()